# Homework 4 Part 2

**Due: Monday, November 9, 11:59 PM**

* This is a group assignment (final project team group).

# Create Your Repository

Create the repo for this assignment by visiting the following link:

https://classroom.github.com/g/m2K_Ea-V

# Data Set

In this assignment, you will be working with the [training data set for the final project](https://ufl.instructure.com/courses/404363/files/folder/Final%20Project/Training%20Data):

* *data_training.npy*
* *labels_training.npy*

**In this homework, you are allowed to use *any* libraries.**

# Computing Resources

Make full use of the computing resources in the [HiPerGator](https://help.rc.ufl.edu/doc/UFRC_Help_and_Documentation).

* Our group is called **"eee4773"**.
* Refer to the lecture recording from Monday, November 2, for information on how to use the HiPerGator.

An alternative is to use the public GPU/TPU resources from [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb).

# Communication

Make sure you communicate often with your team, and you are all following your work plan. You can schedule Zoom meetings using your UFL account: https://ufl.zoom.us/

# Problem 1 (5 points)

Get the team together and discuss rules for working in a group. Design at least 3 rules. 

* I will gather information from all groups and share the consensus group rules.
* For the final project submission (code implementation and report), we will use these group rules as a weight contribution for each team member.
    * For example, suppose that the average score (includes peer- and self-evaluation according to the rules) is 90%, than your individual project grade will be $0.90 \times \text{final project grade}$.
    * All fully contributing members, should expect to get 100% on the participation weight score.

Rules for working in a group:
1. Respond to relevant group messages in a timely manner so everyone in the group is on the same page.
2. Group members should finish assigned parts on time.
3. If a member has trouble finishing the assigned part, he should seek for help from other members as early as possible to allow more time to solve the problem collaboratively.
4. Work assignment is not rigid and can be adjusted based on the difficulty encountered when working on it.

# Problem 2 (10 points)

In a standard supervised Machine Learning algorithm, we typically extract features from data and utilize those features to train a model.

In this problem, you will be exploring the data set and extracting features from the audio signals. A popular feature for speech signals is the Mel Frequency Cepstral Coefficient (MFCC).

**Design at least 2 features.** Compute these features for the training data and build your feature matrix.

* Tip: ```librosa``` continues a few tools to extract features from audio, though it is not required for use in this problem.

(Note: there are multiple solutions to this problem. Grading will be based on critical reasoning and code implementation.)

In [1]:
import numpy as np
import librosa, librosa.display
from sklearn.metrics import accuracy_score

In [2]:
# load data
# 2400 x 100000
data_training = np.load('data_training.npy')
# 1 - 8 (2400 x 1)
labels_training = np.load('labels_training.npy')
# 1 or 2 (2400 x 1)
statements_training = np.load('statements_training.npy')

# label is from 1-8, so I added a place holder at position 0
labels_names = ['place holder', 'neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprise']

In [3]:
# MFCC
sr = 44100
n_mfcc = 13
n_fft_mfcc = 2048
hop_length_mfcc = 512

MFCC_feature_matrix = []
for i in range(data_training.shape[0]):
    MFCC_feature_matrix += [librosa.feature.mfcc(data_training[i], sr = sr, n_mfcc = n_mfcc, n_fft = n_fft_mfcc, hop_length = hop_length_mfcc)]
MFCC_feature_matrix = np.array(MFCC_feature_matrix)
MFCC_feature_matrix = np.mean(MFCC_feature_matrix, axis=2)
print(MFCC_feature_matrix.shape)

(2400, 13)


In [4]:
# STFT
n_fft_stft = 4096
hop_length_stft = 2048

STFT_feature_matrix = []
for i in range(data_training.shape[0]):
    STFT_feature_matrix += [np.abs(librosa.core.stft(data_training[i], n_fft = n_fft_stft, hop_length = hop_length_stft))]
STFT_feature_matrix = np.array(STFT_feature_matrix)
STFT_feature_matrix = np.mean(STFT_feature_matrix, axis=2)
print(STFT_feature_matrix.shape)

(2400, 2049)


In [5]:
# MELSPECTOGRAM
MEL_feature_matrix = []
for i in range(data_training.shape[0]):
    MEL_feature_matrix += [librosa.feature.melspectrogram(data_training[i])]
MEL_feature_matrix = np.array(MEL_feature_matrix)
MEL_feature_matrix = np.mean(MEL_feature_matrix, axis=2)
print(MEL_feature_matrix.shape)

(2400, 128)


In [6]:
# CHROMA
CHROMA_feature_matrix = []
for i in range(data_training.shape[0]):
    CHROMA_feature_matrix += [librosa.feature.chroma_stft(data_training[i])]
CHROMA_feature_matrix = np.array(CHROMA_feature_matrix)
CHROMA_feature_matrix = np.mean(CHROMA_feature_matrix, axis=2)
print(CHROMA_feature_matrix.shape)

(2400, 12)


In [7]:
# ZERO CROSSING RATE
ZCC_feature_matrix = []
for i in range(data_training.shape[0]):
    ZCC_feature_matrix += [librosa.feature.zero_crossing_rate(data_training[i])]
ZCC_feature_matrix = np.array(ZCC_feature_matrix)
ZCC_feature_matrix = np.mean(ZCC_feature_matrix, axis=2)
print(ZCC_feature_matrix.shape)

(2400, 1)


In [8]:
# ROOT MEAN SQUARE ENERGY
RMSE_feature_matrix = []
for i in range(data_training.shape[0]):
    RMSE_feature_matrix += [librosa.feature.rms(data_training[i])]
RMSE_feature_matrix = np.array(RMSE_feature_matrix)
RMSE_feature_matrix = np.mean(RMSE_feature_matrix, axis=2)
print(RMSE_feature_matrix.shape)

(2400, 1)


In [9]:
# SPECTRAL ROLLOFF
SRO_feature_matrix = []
for i in range(data_training.shape[0]):
    SRO_feature_matrix += [librosa.feature.spectral_rolloff(data_training[i])]
SRO_feature_matrix = np.array(SRO_feature_matrix)
SRO_feature_matrix = np.mean(SRO_feature_matrix, axis=2)
print(SRO_feature_matrix.shape)

(2400, 1)


# Problem 3 (30 points)

Implement the $k$-NN and SVM classifiers on your training data. You can choose to apply the classifiers:
* directly on the input data,
* using the features you designed in problem 1,
* both input space and feature space, as to compare both performances.

In your implementation, you should follow the standard procedures:
1. Pre-processing the data, this includes data normalization.
2. Split data into training and test sets. Make sure you use the same training set in both classifiers.
3. Perform $k$-fold Cross-Validation to determine the best set of parameters for each classifier. Consider using the [```GridSearchCV```](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) approach in experimental design.
4. Report the results on the test set using appropriate evaluation metrics.
5. Include a paragraph that compares the performance of both classifiers, and steps you will take to improve upon these preliminary results.

Note: You may include a paragraph description about your experience in using the HiPerGator. If you encounter any issues, please describe them.

In [19]:
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd

#FeatureMatrix = np.concatenate((MFCC_feature_matrix, STFT_feature_matrix, MEL_feature_matrix, CHROMA_feature_matrix, ZCC_feature_matrix, RMSE_feature_matrix, SRO_feature_matrix), axis=1)
FeatureMatrix = np.concatenate((MFCC_feature_matrix, STFT_feature_matrix), axis=1)
print(FeatureMatrix.shape)

Y = np.vstack((labels_training, statements_training)).T
X_train, X_test, y_train, y_test = train_test_split(FeatureMatrix, Y, test_size=0.20)

(2400, 2062)


In [60]:
# SVM
from sklearn.svm import SVC # "Support vector classifier"

#k-FOLD CROSS VALIDATION
params_svm = {
    "kernel" : ["poly", "rbf"],
    "C" : list(range(20000, 30000, 100))
}
GSCV_svm1 = GridSearchCV(SVC(), params_svm, verbose = 3, cv = 3, n_jobs = -1)
GSCV_svm2 = GridSearchCV(SVC(), params_svm, verbose = 3, cv = 3, n_jobs = -1)

In [61]:
Results_label = GSCV_svm1.fit(X_train, y_train[:,0])
Results_statements = GSCV_svm2.fit(X_train, y_train[:,1])

Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   17.2s
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 488 tasks      | elapsed: 10.7min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed: 12.9min finished


Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 488 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed: 10.3min finished


In [62]:
print("Statement 1 Best: {} with parameters {}".format(Results_label.best_score_, Results_label.best_params_ ))
print("Statement 2 Best: {} with parameters {}".format(Results_statements.best_score_, Results_statements.best_params_ ))

Statement 1 Best: 0.5578124999999999 with parameters {'C': 28400, 'kernel': 'rbf'}
Statement 2 Best: 0.8250000000000001 with parameters {'C': 20500, 'kernel': 'rbf'}


In [64]:
print("Training Label Classifier...")
model_s1 = SVC(kernel=Results_label.best_params_['kernel'], 
            C=Results_label.best_params_['C'])
model_s1.fit(X_train, y_train[:,0])
pred_mfcc_svm_s1 = model_s1.predict(X_test)
print("Label Classifier score: ", accuracy_score(y_test[:,0], pred_mfcc_svm_s1))

print("Training Statement Classifier...")
model_s2 = SVC(kernel=Results_statements.best_params_['kernel'], 
            C=Results_statements.best_params_['C'])
model_s2.fit(X_train, y_train[:,1])
pred_mfcc_svm_s2 = model_s2.predict(X_test)
print("Statement Classifier score: ", accuracy_score(y_test[:,1], pred_mfcc_svm_s2))

Training Label Classifier...
Label Classifier score:  0.6208333333333333
Training Statement Classifier...
Statement Classifier score:  0.84375


SVM did slightly better than k-NN did. In this homework, we trained multiple models using different features to classify both the emotion and statement labels. In the future, we will combine both features (MFCC and STFT) and maybe use statement label as one of our features to train a single model that does the classification.

## Submit your Solution

Confirm that you've successfully completed the assignment.

Along with the Notebook, include a PDF of the notebook with your solutions.

```add``` and ```commit``` the final version of your work, and ```push``` your code to your GitHub repository.

Submit the URL of your GitHub Repository as your assignment submission on Canvas. 
* **Only one team member should submit the group repository URL to Canvas.**